In [ ]:
import tensorflow as tf

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Your machine learning code using TensorFlow

In [ ]:
!nvidia-smi

In [ ]:
!pip install scikit-plot

In [ ]:
# Define paths
zip_file_path = '/content/CK+48.zip'  # Path to the uploaded zip file in Colab
extracted_dir_path = '/content/extracted/'  # Adjust the target directory in Colab

# Unzip the file
!unzip -q "$zip_file_path" -d "$extracted_dir_path"

print("Extraction completed.")

In [ ]:
import os
import cv2
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG19
import matplotlib.pyplot as plt

In [ ]:
# Mapping of emotions to numerical labels
emotion_to_label = {'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4, 'sadness': 5, 'surprise': 6}
label_to_text = {0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happy', 5: 'sadness', 6: 'surprise'}




INPUT_PATH = "/content/extracted/CK+48/"

# Count the total number of images
total_images = sum(len(os.listdir(os.path.join(INPUT_PATH, dir_))) for dir_ in emotion_to_label.keys())

# Initialize arrays for images and labels
img_arr = np.empty(shape=(total_images, 224, 224, 3))
img_label = np.empty(shape=(total_images))

# Initialize counters for indexing
idx = 0

for emotion, label_value in emotion_to_label.items():
    dir_path = os.path.join(INPUT_PATH, emotion)
    for f in os.listdir(dir_path):
        img_path = os.path.join(dir_path, f)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (224, 224))
        img_arr[idx] = img_resized
        img_label[idx] = label_value
        idx += 1

In [ ]:
# Build the VGG19 model with fine-tuning
def build_vgg19(input_shape, num_classes):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

    # Fine-tuning: unfreeze some top layers
    for layer in base_model.layers[:-8]:  # Unfreeze last 8 layers
        layer.trainable = True

    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)  # Add dropout for regularization
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x, name="VGG19_emotion")
    return model

# Define batch size and number of epochs
batch_size = 32
epochs = 80

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_datagen.fit(X_train)

In [ ]:
# Build the VGG19 model
INPUT_SHAPE = (224, 224, 3)
model = build_vgg19(input_shape=INPUT_SHAPE, num_classes=len(label_to_text))

# Compile the model
optim = Adam(0.0001)  # Lower learning rate
model.compile(
    loss='categorical_crossentropy',
    optimizer=optim,
    metrics=['accuracy']
)

# Define EarlyStopping and ReduceLROnPlateau callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=15,  # Increased patience
    verbose=1,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    min_delta=0.0001,
    factor=0.4,
    patience=8,  # Increased patience
    min_lr=1e-7,
    verbose=1
)

callbacks = [early_stopping, lr_scheduler]

# Train the model
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    callbacks=callbacks,
    use_multiprocessing=True
)

In [ ]:
# Plot training history
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get predictions from the trained model
y_pred = model.predict(X_test)
predicted_emotions = np.argmax(y_pred, axis=1)

# Create a histogram
plt.hist(predicted_emotions, bins=np.arange(len(label_to_text) + 1), edgecolor='black', alpha=0.7)
plt.xticks(range(len(label_to_text)), label_to_text.values(), rotation=45)
plt.xlabel('Emotion')
plt.ylabel('Frequency')
plt.title('Histogram of Predicted Emotions')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Create a confusion matrix
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), predicted_emotions)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_to_text.values(),
            yticklabels=label_to_text.values())
plt.xlabel('Predicted Emotion')
plt.ylabel('Actual Emotion')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Calculate precision, recall, F1-score
print(classification_report(np.argmax(y_test, axis=1), predicted_emotions,
                            target_names=label_to_text.values()))

In [ ]:
# Calculate ROC curves and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'ROC curve of class {label_to_text[i]} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()








In [ ]:
import numpy as np
from scipy.stats import ttest_rel

# Replace 'other_model_val_accuracy' with the accuracy values of another model
other_model_val_accuracy = [...]  # Replace this with actual accuracy values

# Make sure both arrays have the same length
max_length = max(len(history.history['val_accuracy']), len(other_model_val_accuracy))
history_val_accuracy_padded = np.pad(history.history['val_accuracy'], (0, max_length - len(history.history['val_accuracy'])), mode='constant', constant_values=np.nan)
other_model_val_accuracy_padded = np.pad(other_model_val_accuracy, (0, max_length - len(other_model_val_accuracy)), mode='constant', constant_values=np.nan)

# Perform paired t-test
t_statistic, p_value = ttest_rel(history_val_accuracy_padded, other_model_val_accuracy_padded)

# Define the significance level (alpha)
alpha = 0.05

# Compare the p-value with the significance level
if p_value < alpha:
    print("The difference in performance is statistically significant.")
else:
    print("The difference in performance is not statistically significant.")






In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy over epochs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss over epochs
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()
